In [1]:
import numpy as np
import pandas as pd
import cobra.test
import os

from os.path import join

from time import time


from cobra.flux_analysis import (
    single_gene_deletion, single_reaction_deletion, double_gene_deletion,
    double_reaction_deletion)

import cobra
from cobra.test import create_test_model
cobra_config = cobra.Configuration()
cobra_config.solver = "gurobi"
model = create_test_model("textbook")
model.solver

a = cobra.io.read_sbml_model("iEC1344_C.xml")


Academic license - for non-commercial use only - expires 2024-03-07
Using license file /home/schwartzlab/gurobi.lic


In [2]:
int_reactions= list()
i= -1
n= 0
r = a.reactions[0]
r_id = r.id
zero= list()
one= list()
one_at_zero= list()

for r in a.reactions:  #finds reactions that have one feasible value at flux=0 or for another flux
    i= i+1
    r = a.reactions[i]
    r_id = r.id

    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
    v= len(csvFile["Biomass"])-csvFile["Biomass"].isna().sum()
    n= int((len(csvFile)-1)/2)

    flt= float(csvFile.iloc[n,  2])
    
    if v == 1 and csvFile.iloc[n,  2] != flt:
        one.append(r_id)
    elif v == 0:
        zero.append(r_id)
        
    if v == 1 and csvFile.iloc[n,  2] == flt:
        one_at_zero.append(r_id)

print(one)
print(zero)
print(one_at_zero)

D= pd.DataFrame (one, columns = ['1 feasible value'])
D_at_zero= pd.DataFrame (one_at_zero, columns = ['1 feasible value_at_zero'])


['LCTStex']
[]
['EX_cm_e', 'EX_cobalt2_e', 'EX_colipa_e', 'EX_colipap_e', 'EX_crn_e', 'EX_crn__D_e', 'DM_4crsol_c', 'EX_cu2_e', 'DM_5drib_c', 'EX_cyan_e', 'DM_aacald_c', 'DM_amob_c', 'DM_mththf_c', 'EX_glyb_e', 'EX_gthox_e', 'BIOMASS_Ec_iJO1366_WT_53p95M', 'BIOMASS_Ec_iJO1366_core_53p95M', 'EX_14glucan_e', 'EX_h2o2_e', 'EX_hacolipa_e', 'EX_halipa_e', 'EX_34dhpac_e', 'EX_hg2_e', 'EX_4hoxpacd_e', 'EX_5mtr_e', 'EX_cys__D_e', 'EX_acgal_e', 'EX_inost_e', 'EX_k_e', 'EX_kdo2lipid4_e', 'EX_acolipa_e', 'EX_adocbl_e', 'EX_lipa_e', 'EX_lipa_cold_e', 'EX_ag_e', 'EX_lipoate_e', 'EX_ala_B_e', 'EX_malthx_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_maltttr_e', 'EX_arbt_e', 'EX_meoh_e', 'EX_met__D_e', 'EX_aso3_e', 'EX_met__L_e', 'EX_metsox_R__L_e', 'EX_metsox_S__L_e', 'EX_mg2_e', 'EX_mincyc_e', 'EX_btn_e', 'EX_mmet_e', 'EX_mn2_e', 'EX_ca2_e', 'EX_cbi_e', 'EX_mobd_e', 'EX_n2o_e', 'EX_na1_e', 'EX_nac_e', 'EX_cbl1_e', 'EX_cd2_e', 'EX_chol_e', 'EX_cl_e', 'EX_ni2_e', 'EX_nmn_e', 'EX_dms_e', 'EX_novbcn_e', 'EX_o16

In [3]:
i= 0
r = a.reactions[0]
l2= list()
l3= list()

for r in a.reactions:            #separates reactions that have biomass values for positive and negative flux values
    
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    i3= int(((len(csvFile)-1)/2)+1)

    while i3< len(csvFile):
        inf= csvFile.iloc[i3,  2]
        flt= float(csvFile.iloc[i3,  2])
    
        if csvFile.iloc[i3,  2] == flt and r_id not in l2:
            l2.append(r_id)     #l2 contains reactions that have biomass values for positive flux values
        i3= i3+1
    
    i4= 0
    
    while i4< int(((len(csvFile)-1)/2)):
        inf= csvFile.iloc[i4,  2]
        flt= float(csvFile.iloc[i4,  2])
    
        if csvFile.iloc[i4,  2] == flt and r_id not in l3:
            l3.append(r_id)    #l3 contains reactions that have biomass values for negative flux values
        i4= i4+1

    i= i+1

D1= pd.DataFrame (l2, columns = ['Reaction names'])
D2= pd.DataFrame (l3, columns = ['Reaction names'])
b= D1.iloc[:, 0]
c= D2.iloc[:, 0]

union = pd.Series(np.union1d(b, c))
intersect = pd.Series(np.intersect1d(b, c))
bigb2= union[~union.isin(intersect)]

D3= pd.DataFrame (bigb2, columns = ['Reaction names'])
D3
D3.to_csv("res2.csv")      #contains reactions that have biomass values for either positive or negative flux values


In [4]:
i= 0
indecr_and_stable_above_zero= list()
diff_indecr_above_zero= list()
flat_above_zero= list()
one_to_three_data_point_above_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat above zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    
    my_series = d['Biomass'].squeeze()


    if len(d)> 30 and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_above_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l2 and r_id not in l3:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 2 and count< len(d)-3:
            indecr_and_stable_above_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_above_zero.append(r_id)
        
        elif count> 2 and len(d) < 6:
            flat_above_zero.append(r_id)
        
        elif count> len(d)-3 and len(d)> 5:
            flat_above_zero.append(r_id)
        
        elif len(d)<4:
            one_to_three_data_point_above_zero.append(r_id)
        
        else:
            diff_indecr_above_zero.append(r_id)
    
    i= i+1

flat_above_zero
diff_indecr_above_zero
indecr_and_stable_above_zero


['FRD3', 'O2tex', 'SPODM']

In [5]:
i= 0
indecr_and_stable_below_zero= list()
diff_indecr_below_zero= list()
flat_below_zero= list()
one_to_three_data_point_below_zero= list()
r_id = a.reactions[0]


for r_id in a.reactions:    #separates everything that has an increasing/decreasing pattern from everything that has a saturating pattern and everything that is flat below zero
    l= list()
    n=0
    n2=0
    count=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    d= pd.DataFrame (l, columns = ['Biomass'])
    d.Biomass = d.Biomass.round(2)
    
    my_series = d['Biomass'].squeeze()
    

    if len(d)> 30 and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 0.05*len(d) and count< 0.95*len(d):
            indecr_and_stable_below_zero.append(r_id)

        elif count> 0.95*len(d):
            flat_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)

    
    if len(d)> 1 and len(d)<30  and r_id in l3 and r_id not in l2:
        
        while n2 < len(d):

            if d.iloc[n2,  0] > max(d["Biomass"])-0.01 and d.iloc[n2,  0] < max(d["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        if count> 2 and count< len(d)-3:
            indecr_and_stable_below_zero.append(r_id)

        elif count> 1 and len(d) < 3:
            flat_below_zero.append(r_id)
        
        elif count> 2 and len(d) < 6:
            flat_below_zero.append(r_id)
        
        elif count> len(d)-3 and len(d)> 5:
            flat_below_zero.append(r_id)
        
        elif len(d)<4:
            one_to_three_data_point_below_zero.append(r_id)
        
        else:
            diff_indecr_below_zero.append(r_id)
    
    i= i+1

flat_below_zero
diff_indecr_below_zero
indecr_and_stable_below_zero


['EX_csn_e',
 'EX_glcur_e',
 'EX_gln__L_e',
 'EX_glyc2p_e',
 'EX_h2_e',
 'EX_acgam1p_e',
 'EX_man_e',
 'EX_manglyc_e',
 'EX_o2_e',
 'EX_ppa_e',
 'EX_ppal_e',
 'EX_ppt_e',
 'EX_psclys_e',
 'EX_r5p_e',
 'EX_fald_e',
 'EX_uacgam_e',
 'EX_udpg_e',
 'EX_udpgal_e',
 'EX_fru_e',
 'EX_frulys_e',
 'EX_fruur_e',
 'EX_g1p_e',
 'EX_gal_bD_e',
 'EX_gal1p_e',
 'ACOLIPAabctex',
 'EX_galctn__D_e',
 'EX_galctn__L_e',
 'EX_gam_e',
 'EX_gam6p_e',
 'CHLabcpp',
 'CLIPAabctex',
 'COLIPAabctex',
 'CU1abcpp',
 'DMSOR1',
 'DMSOR1pp',
 'DMSOR2',
 'DMSOR2pp',
 'ECA4COLIPAabctex',
 'DSBAO2',
 'ENLIPAabctex',
 'FUMt2_3pp',
 'ASPO3',
 'ASPO4',
 'ASPO5',
 'GLYBabcpp',
 'HYD2pp',
 'HYD3pp',
 'HG2abcpp',
 'LIPAabctex',
 'NO3R1bpp',
 'NO3R1pp',
 'NO3R2bpp',
 'NTRIR2x',
 'NTRIR3pp',
 'NTRIR4pp',
 'QMO2',
 'TMAOR1',
 'TMAOR1pp',
 'TMAOR2',
 'TMAOR2pp',
 'XYLabcpp',
 'PPPGO',
 'DKMPPD']

In [6]:
i2= 0
mon_incr_above_zero= list()
mon_decr_above_zero= list()

if len(diff_indecr_above_zero)> 0:
    
    for r_id in diff_indecr_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_above_zero)
set2= set(mon_decr_above_zero)
set3= set(mon_incr_above_zero)
inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_above_zero


['ADSL2r',
 'OMPDC',
 'MCOATA',
 'ARGSL',
 'UAGPT3',
 'UAMAS',
 'P5CR',
 'SULR',
 'ARGSS',
 'ACPPAT161',
 'ACGS',
 'SO4tex',
 'CTPS2',
 'ALAR',
 'O2tpp',
 'PAPPT3',
 'PGCD',
 'UMPK',
 'ALAALAr',
 'SERAT',
 'G6PDH2r',
 'GMPS2',
 'HCO3E',
 'COBALT2tpp',
 'ASPCT',
 'PSSA161',
 'PE161abcpp',
 'PSERT',
 'PSP_L',
 'NH4tpp',
 'PSD160',
 'AGPAT161',
 'BPNT',
 'PMPK',
 'NH4tex',
 'ACGK',
 'APG3PAT160',
 'NDPK4',
 'OCBT',
 'UAMAGS',
 'ICDHyr',
 'PE160abcpp',
 'GK1',
 'AGPAT160',
 'NNATr',
 'ADSK',
 'SADT2',
 'ACONTa',
 'PRAGSr',
 'DTMPK',
 'UAGDP',
 'PSD161',
 'G5SADs',
 'TMPK',
 'CS',
 '3OAR140',
 'THRD_L',
 'DASYN161',
 'NADK',
 'UAGCVT',
 'GF6PTA',
 'PSSA160',
 'ASPK',
 'G1PACT',
 'NADS1',
 'UAPGR',
 'GLUPRT',
 'UDCPDP',
 'HSK',
 'ADSS',
 'UAAGDS',
 'PRFGS',
 'ACONTb',
 'ACPPAT160',
 'NDPK2',
 'THRS',
 '3OAS140',
 'DASYN160',
 'ACCOAC',
 'AICART',
 'APG3PAT161',
 'CU2tpp',
 'ADSL1r',
 'PRAIS',
 'GLNS',
 'PRASCSi',
 'UGMDDS',
 'ACODA',
 'CYSS',
 'AIRC2']

In [7]:
w= list()
w2= list()
i2=0

if len(inANDdecr_above_zero)> 0:
    
    while i2 < len(inANDdecr_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2] < max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3]<max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

        i2= i2+1

w


['ADSL2r',
 'OMPDC',
 'MCOATA',
 'ARGSL',
 'UAGPT3',
 'UAMAS',
 'P5CR',
 'SULR',
 'ARGSS',
 'ACPPAT161',
 'ACGS',
 'SO4tex',
 'CTPS2',
 'ALAR',
 'PAPPT3',
 'PGCD',
 'UMPK',
 'ALAALAr',
 'SERAT',
 'G6PDH2r',
 'GMPS2',
 'HCO3E',
 'COBALT2tpp',
 'ASPCT',
 'PSSA161',
 'PE161abcpp',
 'PSERT',
 'PSP_L',
 'PSD160',
 'AGPAT161',
 'BPNT',
 'PMPK',
 'NH4tex',
 'ACGK',
 'APG3PAT160',
 'NDPK4',
 'OCBT',
 'UAMAGS',
 'ICDHyr',
 'PE160abcpp',
 'GK1',
 'AGPAT160',
 'NNATr',
 'ADSK',
 'SADT2',
 'ACONTa',
 'PRAGSr',
 'DTMPK',
 'UAGDP',
 'PSD161',
 'G5SADs',
 'TMPK',
 'CS',
 '3OAR140',
 'THRD_L',
 'DASYN161',
 'NADK',
 'UAGCVT',
 'GF6PTA',
 'PSSA160',
 'ASPK',
 'G1PACT',
 'NADS1',
 'UAPGR',
 'GLUPRT',
 'UDCPDP',
 'HSK',
 'ADSS',
 'UAAGDS',
 'PRFGS',
 'ACONTb',
 'ACPPAT160',
 'NDPK2',
 'THRS',
 '3OAS140',
 'DASYN160',
 'ACCOAC',
 'AICART',
 'APG3PAT161',
 'CU2tpp',
 'ADSL1r',
 'PRAIS',
 'GLNS',
 'PRASCSi',
 'UGMDDS',
 'ACODA',
 'CYSS',
 'AIRC2']

In [8]:
union = pd.Series(np.union1d(w, w2))
unclassified_above_zero = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_above_zero)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_above_zero)
set3= set1-set2
w2= list(set3)

unclassified_above_zero= list(unclassified_above_zero)
unclassified_above_zero


['ASPCT', 'OMPDC']

In [9]:
set1= set(inANDdecr_above_zero)
set2= set(w)
set3= set(unclassified_above_zero)
set4= set1-set2-set3
inANDdecr_above_zero= list(set4)

mon_decr_above_zero= list(mon_decr_above_zero) + list(w)


set1= set(inANDdecr_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_above_zero= list(set3)

mon_incr_above_zero= list(mon_incr_above_zero) + list(w2)


In [10]:
i2= 0
mon_incr_below_zero= list()
mon_decr_below_zero= list()

if len(diff_indecr_below_zero)> 0:
    
    for r_id in diff_indecr_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = diff_indecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero)
set2= set(mon_decr_below_zero)
set3= set(mon_incr_below_zero)
inANDdecr_below_zero= list(set1-set2-set3)
inANDdecr_below_zero


['EX_3hpppn_e',
 'EX_xyl__D_e',
 'EX_so4_e',
 'EX_melib_e',
 'EX_ascb__L_e',
 'EX_all__D_e',
 'EX_dca_e',
 'G3PD2',
 'ASAD',
 'OMMBLHX3',
 'EX_udpacgal_e',
 'EX_lyx__L_e',
 'RPI',
 'PGAMT',
 'ACOTA',
 'EX_lcts_e',
 'EX_tartr__L_e',
 'EX_fe3dcit_e',
 'GLUR',
 'EX_galt_e',
 'EX_LalaDglu_e',
 'EX_galct__D_e',
 'EX_progly_e',
 'EX_nh4_e',
 'EX_ttdcea_e',
 'EX_sbt__D_e',
 'EX_hdca_e',
 'EX_ddca_e',
 'AIRC3',
 'EX_rib__D_e',
 'EX_pppn_e',
 'EX_acmum_e',
 'EX_arab__L_e',
 'EX_acnam_e',
 'EX_acgam_e',
 'OMPHHX3',
 'EX_3hcinnm_e',
 'EX_mnl_e',
 'AGPR',
 'OPHHX3',
 'EX_LalaDgluMdap_e',
 'ORPT',
 'ASPTA',
 'EX_sucr_e',
 'EX_LalaDgluMdapDala_e',
 'EX_hdcea_e',
 'EX_tartr__D_e',
 'EX_ttdca_e',
 'EX_23dappa_e',
 'EX_23ccmp_e',
 'EX_ethso3_e',
 'DHORTS',
 'EX_malt_e',
 'EX_LalaLglu_e',
 'EX_glcr_e',
 'EX_ocdcea_e',
 'EX_chtbs_e',
 'EX_octa_e',
 'IMPC',
 'EX_xylu__L_e',
 'EX_gthrd_e',
 'EX_tre_e',
 'EX_acmana_e',
 'HSDy',
 'EX_fum_e',
 'EX_butso3_e',
 'EX_but_e',
 'EX_galur_e',
 'EX_26dap__M_e',
 'EX_

In [11]:
w= list()
w2= list()
i2=0

if len(inANDdecr_below_zero)> 0:
    
    while i2 < len(inANDdecr_below_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


['ORPT', 'EX_LalaDgluMdapDala_e', 'DHORTS']

In [12]:
union = pd.Series(np.union1d(w, w2))
unclassified_below_zero = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_below_zero)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_below_zero)
set3= set1-set2
w2= list(set3)

unclassified_below_zero= list(unclassified_below_zero)
unclassified_below_zero


['DHORTS', 'ORPT']

In [13]:
set1= set(inANDdecr_below_zero)
set2= set(w)
set3= set(unclassified_below_zero)
set4= set1-set2-set3
inANDdecr_below_zero= list(set4)

mon_decr_below_zero= list(mon_decr_below_zero) + list(w)


set1= set(inANDdecr_below_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_below_zero= list(set3)

mon_incr_below_zero= list(mon_incr_below_zero) + list(w2)


In [14]:
i2= 0
mon_incr_and_stable_above_zero= list()
mon_decr_and_stable_above_zero= list()

if len(indecr_and_stable_above_zero)> 0:
    
    for r_id in indecr_and_stable_above_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend above zero
    
        i=0
        r_id = indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_above_zero)
inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_above_zero


['O2tex']

In [15]:
w= list()
w2= list()
i2=0

if len(inANDdecr_and_stable_above_zero)> 0:
    
    while i2 < len(inANDdecr_and_stable_above_zero):        #this code finds reactions that are not truly both increasing and decreasing and stable above zero
    
        i=0
        r_id = inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w


[]

In [16]:
union = pd.Series(np.union1d(w, w2))
unclassified_above_zero2 = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_above_zero2)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_above_zero2)
set3= set1-set2
w2= list(set3)

unclassified_above_zero= list(unclassified_above_zero2)+list(unclassified_above_zero)
unclassified_above_zero


['ASPCT', 'OMPDC']

In [17]:
set1= set(inANDdecr_and_stable_above_zero)
set2= set(w)
set3= set(unclassified_above_zero2)
set4= set1-set2-set3
inANDdecr_and_stable_above_zero= list(set4)

mon_decr_and_stable_above_zero= list(mon_decr_and_stable_above_zero) + list(w)


set1= set(inANDdecr_and_stable_above_zero)
set2= set(w2)
set3= set1-set2
inANDdecr_and_stable_above_zero= list(set3)

mon_incr_and_stable_above_zero= list(mon_incr_and_stable_above_zero) + list(w2)


In [18]:
i2= 0
mon_incr_and_stable_below_zero= list()
mon_decr_and_stable_below_zero= list()

if len(indecr_and_stable_below_zero)> 0:
    
    for r_id in indecr_and_stable_below_zero:                    #this code separates reactions with an increasing and stable trend from reactions with a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero.append(r_id)

        i2= i2 + 1

set1= set(indecr_and_stable_below_zero)
set2= set(mon_decr_and_stable_below_zero)
set3= set(mon_incr_and_stable_below_zero)
inANDdecr_and_stable_below_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero


['NTRIR3pp',
 'NTRIR2x',
 'EX_galctn__D_e',
 'EX_udpgal_e',
 'NO3R1bpp',
 'EX_manglyc_e',
 'EX_fru_e',
 'EX_ppal_e',
 'EX_o2_e',
 'EX_uacgam_e',
 'PPPGO',
 'EX_g1p_e',
 'EX_udpg_e',
 'EX_h2_e',
 'NTRIR4pp',
 'EX_ppa_e',
 'EX_galctn__L_e',
 'EX_man_e',
 'EX_gam_e']

In [19]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero):       #this code finds reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w2.append(r_id)

    
        i2= i2+1

w3


[]

In [20]:
union = pd.Series(np.union1d(w, w2))
unclassified_below_zero2 = pd.Series(np.intersect1d(w, w2))

set1= set(w)
set2= set(unclassified_below_zero2)
set3= set1-set2
w= list(set3)

set1= set(w2)
set2= set(unclassified_below_zero2)
set3= set1-set2
w2= list(set3)

unclassified_below_zero= list(unclassified_below_zero2)+list(unclassified_below_zero)
unclassified_below_zero


['DHORTS', 'ORPT']

In [21]:
set1= set(inANDdecr_and_stable_below_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero2)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero= list(set5)

mon_decr_and_stable_below_zero= list(mon_decr_and_stable_below_zero) + list(w3)
mon_incr_and_stable_below_zero= list(mon_incr_and_stable_below_zero) + list(w4)


In [22]:
i= 0
over_below_zero_and_under_above_zero= list()
under_below_zero_and_over_above_zero= list()
over_below_zero_and_over_above_zero= list()
under_below_zero_and_under_above_zero= list()
r_id = a.reactions[0]

for r_id in a.reactions:    #separates reactions based on whether they have less than 30 or not values both in the above and the below zero interval
    l= list()
    n=0
    n2=0
    count=0
    count2=0
    r = a.reactions[i]
    r_id = r.id
    csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

    while n < int(((len(csvFile)-1)/2)+1):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile2= pd.DataFrame (l, columns = ['Biomass'])
    csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
    n= int((len(csvFile)-1)/2)
    l4= list()
    
    while n < len(csvFile):
        inf= csvFile.iloc[n,  2]
        flt= float(csvFile.iloc[n,  2])
    
        if csvFile.iloc[n,  2] == flt:
            l4.append(csvFile.iloc[n,  2])
        n= n+1

    csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
    csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
    my_series = csvFile2['Biomass'].squeeze()
    my_series2 = csvFile3['Biomass'].squeeze()


    if (len(csvFile2)>30 or len(csvFile2)==30) and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
        
        over_below_zero_and_under_above_zero.append(r_id)
        
    if len(csvFile2)<30 and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        under_below_zero_and_over_above_zero.append(r_id)
    
    if (len(csvFile2)>30 or len(csvFile2)==30) and (len(csvFile3)> 30 or len(csvFile3)==30) and r_id in l3 and r_id in l2:
    
        over_below_zero_and_over_above_zero.append(r_id)
        
    if len(csvFile2)<30 and len(csvFile3)< 30 and r_id in l3 and r_id in l2:
    
        under_below_zero_and_under_above_zero.append(r_id)
    
    
    i= i+1
    
over_below_zero_and_under_above_zero


['EX_glc__D_e',
 'EX_glcn_e',
 'EX_glu__L_e',
 'EX_gly_e',
 'EX_glyald_e',
 'EX_glyc_e',
 'EX_glyc__R_e',
 'EX_glyc3p_e',
 'EX_glyclt_e',
 'EX_gua_e',
 'EX_h_e',
 'EX_12ppd__S_e',
 'EX_4abut_e',
 'EX_hxa_e',
 'EX_hxan_e',
 'EX_5dglcn_e',
 'EX_cys__L_e',
 'EX_ac_e',
 'EX_idon__L_e',
 'EX_acald_e',
 'EX_lac__D_e',
 'EX_lac__L_e',
 'EX_ade_e',
 'EX_agm_e',
 'EX_lys__L_e',
 'EX_akg_e',
 'EX_ala__D_e',
 'EX_ala__L_e',
 'EX_mal__L_e',
 'EX_alaala_e',
 'EX_alltn_e',
 'EX_arg__L_e',
 'EX_asn__L_e',
 'EX_asp__L_e',
 'EX_dha_e',
 'EX_cgly_e',
 'EX_xan_e',
 'EX_orn_e',
 'EX_pro__L_e',
 'EX_ptrc_e',
 'EX_pyr_e',
 'EX_ser__L_e',
 'EX_etha_e',
 'EX_succ_e',
 'EX_etoh_e',
 'EX_thr__L_e',
 'EX_fe2_e',
 'EX_for_e',
 'ACACT8r_copy1',
 'EX_g3pe_e',
 'EX_g3pg_e',
 'EX_gal_e',
 'ACOATA',
 'AKGDH',
 'ADA',
 'ADD',
 'CAT',
 'CMPN',
 'CBPS',
 'DADA',
 'CTECOAI6_copy1',
 'CTECOAI7_copy1',
 'CTECOAI8_copy1',
 'CYSabc2pp',
 'CYTBD2pp',
 'CYTBDpp',
 'CYTBO3_4pp',
 'CYTD',
 'CYTDH',
 'DHAPT',
 'DAAD',
 'DHORD2',
 

In [23]:
i= 0
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_diff_indecr_above_zero= list()
rest_both_positive_and_negative= list()
indecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
flat_below_zero_and_flat_above_zero= list()
diff_indecr_below_zero_and_flat_above_zero= list()
flat_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_flat_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
flat_below_zero_and_indecr_and_stable_above_zero= list()
diff_indecr_below_zero_and_indecr_and_stable_above_zero= list()
indecr_and_stable_below_zero_and_diff_indecr_above_zero= list()
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
flat_below_zero_and_one_to_three_data_point_above_zero= list()
diff_indecr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(over_below_zero_and_under_above_zero)> 0:

    for r_id in over_below_zero_and_under_above_zero:    #separates over_below_zero_and_under_above_zero reactions in more specific subcategories
    
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and len(csvFile3)<4:
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2) and (count2> 2 and count2< len(csvFile3)-3):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 6):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 3:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and len(csvFile3)<4:
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
        elif count< 0.05*len(csvFile2) and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 2 and count2< len(csvFile3)-3):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 3:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif count< 0.05*len(csvFile2) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and (count2> 2 and len(csvFile3) < 6):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['NADH16pp']

In [24]:
i=0
one_to_three_data_point_below_zero_and_diff_indecr_above_zero= list()
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_flat_above_zero= list()

if len(under_below_zero_and_over_above_zero)> 0:

    for r_id in under_below_zero_and_over_above_zero:    #separates under_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 3 and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count<3 and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

            
    
        elif len(csvFile2)<4 and count2< 0.05*len(csvFile3):
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
        
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['NADH16pp']

In [25]:
i=0

if len(over_below_zero_and_over_above_zero)> 0:

    for r_id in over_below_zero_and_over_above_zero:    #separates over_below_zero_and_over_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = over_below_zero_and_over_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and (count2== 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.05*len(csvFile3) or count2> 0.05*len(csvFile3) and count2< 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count<0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 0.05*len(csvFile2) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
   
        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and (count2 == 0.95*len(csvFile3) or count2> 0.95*len(csvFile3)):
            flat_below_zero_and_flat_above_zero.append(r_id)

    
    
        elif (count== 0.05*len(csvFile2) or count> 0.05*len(csvFile2) and count< 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 0.05*len(csvFile2) and count2< 0.05*len(csvFile3):
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count == 0.95*len(csvFile2) or count> 0.95*len(csvFile2)) and count2< 0.05*len(csvFile3):
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['NADH16pp']

In [26]:
i=0
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero= list()

if len(under_below_zero_and_under_above_zero)> 0:
    
    for r_id in under_below_zero_and_under_above_zero:    #separates under_below_zero_and_under_above_zero reactions in more specific subcategories
        l= list()
        n=0
        n2=0
        count=0
        count2=0
        r_id = under_below_zero_and_under_above_zero[i]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))

        while n < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
    
    
        n= int((len(csvFile)-1)/2)
        l4= list()
    
        while n < len(csvFile):
            inf= csvFile.iloc[n,  2]
            flt= float(csvFile.iloc[n,  2])
    
            if csvFile.iloc[n,  2] == flt:
                l4.append(csvFile.iloc[n,  2])
            n= n+1

        csvFile3= pd.DataFrame (l4, columns = ['Biomass'])
        csvFile3.Biomass = csvFile3.Biomass.round(2)
    
    
        my_series = csvFile2['Biomass'].squeeze()
        my_series2 = csvFile3['Biomass'].squeeze()


        while n2 < len(csvFile2):

            if csvFile2.iloc[n2,  0] > max(csvFile2["Biomass"])-0.01 and csvFile2.iloc[n2,  0] < max(csvFile2["Biomass"])+0.01:
                count= count+1
    
            n2= n2+1
        
        n2=0
        while n2 < len(csvFile3):

            if csvFile3.iloc[n2,  0] > max(csvFile3["Biomass"])-0.01 and csvFile3.iloc[n2,  0] < max(csvFile3["Biomass"])+0.01:
                count2= count2+1
    
            n2= n2+1
    
        if len(csvFile2)<4 and len(csvFile3)<4:
            one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

        elif len(csvFile2)<4 and (count2> 2 and count2< len(csvFile3)-3):
            one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            
        elif (count> 2 and count< len(csvFile2)-3) and len(csvFile3)<4:       
            indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> 2 and len(csvFile2) < 6) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
        elif (count> 1 and len(csvFile2) < 3) and len(csvFile3)<4:       
            flat_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        
            
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> 2 and count2< len(csvFile3)-3):        
            indecr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and count2< len(csvFile3)-3):
            flat_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
        elif count< 3 and (count2> 2 and count2< len(csvFile3)-3):
            diff_indecr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 2 and len(csvFile3) < 6):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif len(csvFile2)<4 and (count2> 1 and len(csvFile3) < 3):
            one_to_three_data_point_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> 2 and len(csvFile3) < 6):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and (count2> 1 and len(csvFile3) < 3):
            indecr_and_stable_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> 2 and len(csvFile3) < 6):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)
    
        elif count< 3 and (count2> 1 and len(csvFile3) < 3):
            diff_indecr_below_zero_and_flat_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
        
        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> len(csvFile3)-3 and len(csvFile3)>5):
            flat_below_zero_and_flat_above_zero.append(r_id)

        elif (count> 1 and len(csvFile2) < 3) and (count2> 2 and len(csvFile3) < 6):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and (count2> 1 and len(csvFile3) < 3):
            flat_below_zero_and_flat_above_zero.append(r_id)
    
    
    
        elif len(csvFile2)<4 and count2< 3:
            one_to_three_data_point_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and len(csvFile3)<4:
            diff_indecr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
    
        elif (count> 2 and count< len(csvFile2)-3) and count2< 3:
            indecr_and_stable_below_zero_and_diff_indecr_above_zero.append(r_id)
        
        elif count< 3 and count2< 3:
            diff_indecr_below_zero_and_diff_indecr_above_zero.append(r_id)

        elif (count> len(csvFile2)-3 and len(csvFile2)>5) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 2 and len(csvFile2) < 6) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
        elif (count> 1 and len(csvFile2) < 3) and count2< 3:
            flat_below_zero_and_diff_indecr_above_zero.append(r_id)
    
    
        i= i+1

indecr_and_stable_below_zero_and_indecr_and_stable_above_zero


['NADH16pp']

In [27]:
flat_below_zero_and_flat_above_zero
indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero
indecr_and_stable_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_diff_indecr_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_diff_indecr_above_zero
diff_indecr_below_zero_and_indecr_and_stable_above_zero
diff_indecr_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_diff_indecr_above_zero
flat_below_zero_and_indecr_and_stable_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero
diff_indecr_below_zero_and_flat_above_zero


indecr_and_stable_below_zero_and_indecr_and_stable_above_zero
indecr_and_stable_below_zero_and_flat_above_zero
one_to_three_data_point_above_zero
one_to_three_data_point_below_zero


['EX_pi_e',
 '12PPDRtex',
 '12PPDRtpp',
 '12PPDStex',
 '12PPDStpp',
 '3HPPtex',
 'ABUTtex',
 'ACACCT',
 'ACGAptspp',
 '5DGLCNR',
 '5DGLCNt2rpp',
 'ACMANAptspp',
 '5DGLCNtex',
 'AKGt2rpp',
 'AKGtex',
 'ACSERtex',
 'ALAALAtex',
 'ADEt2rpp',
 'ADEtex',
 'ADNt2pp',
 'AGMtex',
 'CGLYtex',
 'BUTCT',
 'CITtex',
 'CPPPGO',
 'DADNt2pp',
 'DALAtex',
 'DAPabcpp',
 'DAPtex',
 'DCYTt2pp',
 'DDGLK',
 'CYStex',
 'DGSNt2pp',
 'CYTDt2pp',
 'DINSt2pp',
 'ECOAH3',
 'DURIt2pp',
 'ETHAtex',
 'ARGtex',
 'F6Pt6_2pp',
 'FACOAL80t2pp',
 'G1PPpp',
 'G3PEtex',
 'G3PGtex',
 'G6Pt6_2pp',
 'GLCNtex',
 'GAL1PPpp',
 'GALM2pp',
 'GALtex',
 'GAM6Pt6_2pp',
 'GAMptspp',
 'GLUtex',
 'GUAtex',
 'GLYC3Ptex',
 'HXAtex',
 'HYXNtex',
 'HYXNtpp',
 'IDONt2rpp',
 'IDONtex',
 'ILETA',
 'ILEtex',
 'HIStex',
 'INDOLEtex',
 'INSt2pp',
 'HOMtex',
 'IPPMIa',
 'IPPMIb',
 'KARA1',
 'LALADGLUtpp',
 'LALALGLUtpp',
 'LEUtex',
 'LYStex',
 'MANptspp',
 'MAN6Pt6_2pp',
 'METSOX1abcpp',
 'METSOX2abcpp',
 'METabcpp',
 'MNLptspp',
 'NACtpp',
 'GSN

In [28]:
len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+len(indecr_and_stable_below_zero_and_flat_above_zero)+len(one)+len(zero)+len(flat_below_zero_and_flat_above_zero)+len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_flat_above_zero)+len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_diff_indecr_above_zero)+len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_diff_indecr_above_zero)+len(flat_below_zero_and_indecr_and_stable_above_zero)+len(flat_below_zero_and_one_to_three_data_point_above_zero)+len(diff_indecr_below_zero_and_flat_above_zero)+len(one_at_zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(diff_indecr_above_zero)+len(diff_indecr_below_zero)+len(one_to_three_data_point_above_zero)+len(one_to_three_data_point_below_zero)


2726

In [29]:
lt= list(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)+list(indecr_and_stable_below_zero_and_flat_above_zero)+list(one)+list(zero)+list(flat_below_zero_and_flat_above_zero)+list(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(indecr_and_stable_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_flat_above_zero)+list(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)+list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_diff_indecr_above_zero)+list(diff_indecr_below_zero_and_indecr_and_stable_above_zero)+list(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_diff_indecr_above_zero)+list(flat_below_zero_and_indecr_and_stable_above_zero)+list(flat_below_zero_and_one_to_three_data_point_above_zero)+list(diff_indecr_below_zero_and_flat_above_zero)+list(one_at_zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(diff_indecr_above_zero)+list(diff_indecr_below_zero)+list(one_to_three_data_point_above_zero)+list(one_to_three_data_point_below_zero)


In [30]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [31]:
csvFile = pd.read_csv('all reactions lactose.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [32]:
i2= 0
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()
decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list()

if len(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero:                    #this code finds which of the indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        i=0
        r_id = indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
        type(my_series)

        if my_series.is_monotonic_decreasing ==True:
            decr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
        elif my_series.is_monotonic ==True:
            incr_and_stable_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero

set1= set(indecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set3= set(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero


['EX_glc__D_e', 'EX_agm_e']

In [33]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
    
        i2= i2+1

w3


[]

In [34]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_one_to_three_data_point_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_one_to_three_data_point_above_zero= list(unclassified_below_zero_and_one_to_three_data_point_above_zero)
unclassified_below_zero_and_one_to_three_data_point_above_zero


[]

In [35]:
set1= set(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(set5)

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero= list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [36]:
i2= 0
indecr_and_stable_below_zero_and_mon_incr_above_zero= list()
indecr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_diff_indecr_above_zero:                  #this code finds which of the indecr_and_stable_below_zero_and_diff_indecr_above_zero reactions have an increasing trend and which a decreasing trend above zero
    
        r_id = indecr_and_stable_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                indecr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                indecr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_diff_indecr_above_zero)
set2= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
indecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [37]:
w3= list()
w4= list()
i2=0

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(indecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable above zero
    
        i= int((len(csvFile)-1)/2)
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        
        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)

    
        i2= i2+1

w3


[]

In [38]:
union = pd.Series(np.union1d(w3, w4))
indecr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

indecr_and_stable_below_zero_and_unclassified_above_zero= list(indecr_and_stable_below_zero_and_unclassified_above_zero)
indecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [39]:
set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
indecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set5)

indecr_and_stable_below_zero_and_mon_decr_above_zero= list(indecr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
indecr_and_stable_below_zero_and_mon_incr_above_zero= list(indecr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [40]:
i2= 0
decr_and_stable_below_zero_and_mon_decr_above_zero= list()
incr_and_stable_below_zero_and_mon_decr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_mon_decr_above_zero:                #this code finds which of the indecr_and_stable_below_zero_and_mon_decr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_decr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_decr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero


['Kabcpp',
 'SULabcpp',
 'CBPS',
 'NI2uabcpp',
 'MNt2pp',
 'ARGDCpp',
 'H2Otex',
 'PPCK',
 'EX_ac_e',
 'PIuabcpp',
 'ZNabcpp',
 'PPA']

In [41]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_decr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['Kabcpp',
 'SULabcpp',
 'CBPS',
 'NI2uabcpp',
 'MNt2pp',
 'ARGDCpp',
 'PIuabcpp',
 'ZNabcpp',
 'PPA']

In [42]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_decr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_decr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_decr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_decr_above_zero= list(unclassified_below_zero_and_mon_decr_above_zero)
unclassified_below_zero_and_mon_decr_above_zero


[]

In [43]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_decr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_decr_above_zero= list(decr_and_stable_below_zero_and_mon_decr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_decr_above_zero= list(incr_and_stable_below_zero_and_mon_decr_above_zero) + list(w4)


In [44]:
i2= 0
decr_and_stable_below_zero_and_mon_incr_above_zero= list()
incr_and_stable_below_zero_and_mon_incr_above_zero= list()

if len(indecr_and_stable_below_zero_and_mon_incr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_mon_incr_above_zero:              #this code finds which of the indecr_and_stable_below_zero_and_mon_incr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(decr_and_stable_below_zero_and_mon_incr_above_zero)
set3= set(incr_and_stable_below_zero_and_mon_incr_above_zero)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero


[]

In [45]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_mon_incr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [46]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_incr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_incr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_incr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_incr_above_zero= list(unclassified_below_zero_and_mon_incr_above_zero)
unclassified_below_zero_and_mon_incr_above_zero


[]

In [47]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_incr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [48]:
i2= 0
decr_and_stable_below_zero_and_inANDdecr_above_zero= list()
incr_and_stable_below_zero_and_inANDdecr_above_zero= list()

if len(indecr_and_stable_below_zero_and_inANDdecr_above_zero)> 0:

    for r_id in indecr_and_stable_below_zero_and_inANDdecr_above_zero:           #this code finds which of the indecr_and_stable_below_zero_and_inANDdecr_above_zero reactions have an increasing and stable trend and which a decreasing and stable trend below zero
    
        r_id = indecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)


        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                decr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                incr_and_stable_below_zero_and_inANDdecr_above_zero.append(r_id)
    
        i2= i2 + 1


set1= set(indecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(decr_and_stable_below_zero_and_inANDdecr_above_zero)
set3= set(incr_and_stable_below_zero_and_inANDdecr_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero


[]

In [49]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero):       #this code finds inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero reactions that are not truly both increasing and decreasing and stable below zero
    
        i=0
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < ((len(csvFile)-1)/2)+1:
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [50]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_inANDdecr_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_inANDdecr_above_zero= list(unclassified_below_zero_and_inANDdecr_above_zero)
unclassified_below_zero_and_inANDdecr_above_zero


[]

In [51]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_inANDdecr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero= list(set5)

decr_and_stable_below_zero_and_mon_incr_above_zero= list(decr_and_stable_below_zero_and_mon_incr_above_zero) + list(w3)
incr_and_stable_below_zero_and_mon_incr_above_zero= list(incr_and_stable_below_zero_and_mon_incr_above_zero) + list(w4)


In [52]:
i2= 0
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list()
one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list()

if len(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_indecr_and_stable_above_zero)
set2= set(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)
set3= set(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [53]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [54]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [55]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero= list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero) + list(w4)


In [56]:
i2= 0
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list()
one_to_three_data_point_below_zero_and_mon_decr_above_zero= list()

if len(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in one_to_three_data_point_below_zero_and_diff_indecr_above_zero:
    
        r_id = one_to_three_data_point_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                one_to_three_data_point_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                one_to_three_data_point_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(one_to_three_data_point_below_zero_and_diff_indecr_above_zero)
set2= set(one_to_three_data_point_below_zero_and_mon_decr_above_zero)
set3= set(one_to_three_data_point_below_zero_and_mon_incr_above_zero)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
one_to_three_data_point_below_zero_and_inANDdecr_above_zero


['PGL']

In [57]:
w3= list()
w4= list()
i2=0

if len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = one_to_three_data_point_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['PGL']

In [58]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [59]:
w4

set1= set(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
one_to_three_data_point_below_zero_and_inANDdecr_above_zero= list(set5)

one_to_three_data_point_below_zero_and_mon_decr_above_zero= list(one_to_three_data_point_below_zero_and_mon_decr_above_zero) + list(w3)
one_to_three_data_point_below_zero_and_mon_incr_above_zero= list(one_to_three_data_point_below_zero_and_mon_incr_above_zero) + list(w4)


In [60]:
i2= 0
diff_indecr_below_zero_and_mon_incr_above_zero= list()
diff_indecr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_diff_indecr_above_zero:
    
        r_id = diff_indecr_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                diff_indecr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                diff_indecr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_diff_indecr_above_zero)
set2= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set3= set(diff_indecr_below_zero_and_mon_incr_above_zero)
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
diff_indecr_below_zero_and_inANDdecr_above_zero


['MDH',
 'GAPD',
 'ATPS4rpp',
 'EX_co2_e',
 'ENO',
 'FUM',
 'PGI',
 'CYTBO3_4pp',
 'GND']

In [61]:
w3= list()
w4= list()
i2=0

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(diff_indecr_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['MDH', 'FUM', 'PGI', 'GND']

In [62]:
union = pd.Series(np.union1d(w3, w4))
one_to_three_data_point_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

one_to_three_data_point_below_zero_and_unclassified_above_zero= list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero2)
one_to_three_data_point_below_zero_and_unclassified_above_zero


[]

In [63]:
w4

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(one_to_three_data_point_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
diff_indecr_below_zero_and_inANDdecr_above_zero= list(set5)

diff_indecr_below_zero_and_mon_decr_above_zero= list(diff_indecr_below_zero_and_mon_decr_above_zero) + list(w3)
diff_indecr_below_zero_and_mon_incr_above_zero= list(diff_indecr_below_zero_and_mon_incr_above_zero) + list(w4)


In [64]:
i2= 0
mon_decr_below_zero_and_mon_decr_above_zero= list()
mon_incr_below_zero_and_mon_decr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_decr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_decr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_decr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_decr_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_above_zero)
set3= set(mon_incr_below_zero_and_mon_decr_above_zero)
inANDdecr_below_zero_and_mon_decr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_decr_above_zero


['EX_succ_e',
 'EX_alltn_e',
 'EX_glyc_e',
 'EX_glyc__R_e',
 'THD2pp',
 'H2Otpp',
 'FE3abcpp',
 'CO2tex',
 'EX_for_e',
 'PGM',
 'H2SO',
 'PPS',
 'ME2',
 'CITL',
 'PGK',
 'EX_etoh_e',
 'GLYCDx',
 'GLUDy',
 'CAT',
 'FLDR2',
 'FBP',
 'CO2tpp',
 'EX_asn__L_e']

In [65]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_decr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_decr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_decr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

        
        
w3


['PGK']

In [66]:
w4

['ME2', 'CITL', 'GLYCDx', 'GLUDy', 'FLDR2', 'FBP']

In [67]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_decr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_decr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_decr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_decr_above_zero= list(unclassified_below_zero_and_mon_decr_above_zero)+list(unclassified_below_zero_and_mon_decr_above_zero2)
unclassified_below_zero_and_mon_decr_above_zero


[]

In [68]:
w4

set1= set(inANDdecr_below_zero_and_mon_decr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_decr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_mon_decr_above_zero= list(set5)

mon_decr_below_zero_and_mon_decr_above_zero= list(mon_decr_below_zero_and_mon_decr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_decr_above_zero= list(mon_incr_below_zero_and_mon_decr_above_zero) + list(w4)


In [69]:
i2= 0
mon_decr_below_zero_and_mon_incr_above_zero= list()
mon_incr_below_zero_and_mon_incr_above_zero= list()

if len(diff_indecr_below_zero_and_mon_incr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_mon_incr_above_zero:
    
        r_id = diff_indecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_incr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_mon_incr_above_zero)
set2= set(mon_decr_below_zero_and_mon_incr_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_above_zero)
inANDdecr_below_zero_and_mon_incr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_mon_incr_above_zero


[]

In [70]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_mon_incr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_mon_incr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_mon_incr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [71]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_mon_incr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_mon_incr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_mon_incr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_mon_incr_above_zero= list(unclassified_below_zero_and_mon_incr_above_zero)+list(unclassified_below_zero_and_mon_incr_above_zero2)
unclassified_below_zero_and_mon_incr_above_zero


[]

In [72]:
w4

set1= set(inANDdecr_below_zero_and_mon_incr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_mon_incr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_mon_incr_above_zero= list(set5)

mon_decr_below_zero_and_mon_incr_above_zero= list(mon_decr_below_zero_and_mon_incr_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_above_zero= list(mon_incr_below_zero_and_mon_incr_above_zero) + list(w4)


In [73]:
i2= 0
mon_decr_below_zero_and_inANDdecr_above_zero= list()
mon_incr_below_zero_and_inANDdecr_above_zero= list()

if len(diff_indecr_below_zero_and_inANDdecr_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_inANDdecr_above_zero:                  
        
        r_id = diff_indecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_inANDdecr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_inANDdecr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_inANDdecr_above_zero)
set2= set(mon_decr_below_zero_and_inANDdecr_above_zero)
set3= set(mon_incr_below_zero_and_inANDdecr_above_zero)
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_above_zero


[]

In [74]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [75]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_inANDdecr_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_inANDdecr_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_inANDdecr_above_zero= list(unclassified_below_zero_and_inANDdecr_above_zero)+list(unclassified_below_zero_and_inANDdecr_above_zero2)
unclassified_below_zero_and_inANDdecr_above_zero


[]

In [76]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_inANDdecr_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_inANDdecr_above_zero= list(set5)

mon_decr_below_zero_and_inANDdecr_above_zero= list(mon_decr_below_zero_and_inANDdecr_above_zero) + list(w3)
mon_incr_below_zero_and_inANDdecr_above_zero= list(mon_incr_below_zero_and_inANDdecr_above_zero) + list(w4)


In [77]:
i2= 0
mon_decr_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_below_zero_and_indecr_and_stable_above_zero= list()

if len(diff_indecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = diff_indecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_indecr_and_stable_above_zero


[]

In [78]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int(((len(csvFile)-1)/2)+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [79]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_indecr_and_stable_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_indecr_and_stable_above_zero= list(unclassified_below_zero_and_indecr_and_stable_above_zero)
unclassified_below_zero_and_indecr_and_stable_above_zero


[]

In [80]:
w4

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_indecr_and_stable_above_zero= list(set5)

mon_decr_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [81]:
i2= 0
inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [82]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [83]:
union = pd.Series(np.union1d(w3, w4))
inANDdecr_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(inANDdecr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(inANDdecr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

inANDdecr_below_zero_and_unclassified_above_zero= list(inANDdecr_below_zero_and_unclassified_above_zero)
inANDdecr_below_zero_and_unclassified_above_zero


[]

In [84]:
w4

set1= set(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(inANDdecr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [85]:
i2= 0
mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [86]:
w3= list()
w4= list()
i2=0

if len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [87]:
union = pd.Series(np.union1d(w3, w4))
mon_decr_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_decr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_decr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_decr_below_zero_and_unclassified_above_zero= list(mon_decr_below_zero_and_unclassified_above_zero)
mon_decr_below_zero_and_unclassified_above_zero


[]

In [88]:
w4

set1= set(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_decr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_decr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_decr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [89]:
i2= 0
mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero


['EX_h2o_e', 'SUCDi']

In [90]:
w3= list()
w4= list()
i2=0

if len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['SUCDi']

In [91]:
union = pd.Series(np.union1d(w3, w4))
mon_incr_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_incr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_incr_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_incr_below_zero_and_unclassified_above_zero= list(mon_incr_below_zero_and_unclassified_above_zero)
mon_incr_below_zero_and_unclassified_above_zero


[]

In [92]:
w4

set1= set(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_incr_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_incr_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_incr_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [93]:
i2= 0
mon_decr_below_zero_and_one_to_three_data_point_above_zero= list()
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list()

if len(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_one_to_three_data_point_above_zero:
    
        r_id = diff_indecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_one_to_three_data_point_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(mon_decr_below_zero_and_one_to_three_data_point_above_zero)
set3= set(mon_incr_below_zero_and_one_to_three_data_point_above_zero)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_one_to_three_data_point_above_zero


['EX_12ppd__S_e',
 'EX_ade_e',
 'EX_idon__L_e',
 'EX_4abut_e',
 'EX_gua_e',
 'EX_hxa_e',
 'EX_xan_e',
 'EX_ptrc_e',
 'GLCt2pp',
 'EX_ala__D_e',
 'EX_cgly_e',
 'EX_pro__L_e',
 'EX_gal_e']

In [94]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_one_to_three_data_point_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


['EX_ala__D_e']

In [95]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_one_to_three_data_point_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_one_to_three_data_point_above_zero= list(unclassified_below_zero_and_one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_one_to_three_data_point_above_zero2)
unclassified_below_zero_and_one_to_three_data_point_above_zero


[]

In [96]:
w4

set1= set(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_one_to_three_data_point_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_one_to_three_data_point_above_zero= list(set5)

mon_decr_below_zero_and_one_to_three_data_point_above_zero= list(mon_decr_below_zero_and_one_to_three_data_point_above_zero) + list(w3)
mon_incr_below_zero_and_one_to_three_data_point_above_zero= list(mon_incr_below_zero_and_one_to_three_data_point_above_zero) + list(w4)


In [97]:
i2= 0
flat_below_zero_and_mon_decr_above_zero= list()
flat_below_zero_and_mon_incr_above_zero= list()

if len(flat_below_zero_and_diff_indecr_above_zero)>0:
    
    for r_id in flat_below_zero_and_diff_indecr_above_zero:
    
        r_id = flat_below_zero_and_diff_indecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_diff_indecr_above_zero)
set2= set(flat_below_zero_and_mon_decr_above_zero)
set3= set(flat_below_zero_and_mon_incr_above_zero)
flat_below_zero_and_inANDdecr_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_above_zero


[]

In [98]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [99]:
union = pd.Series(np.union1d(w3, w4))
flat_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(flat_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(flat_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

flat_below_zero_and_unclassified_above_zero= list(flat_below_zero_and_unclassified_above_zero)
flat_below_zero_and_unclassified_above_zero


[]

In [100]:
w4

set1= set(flat_below_zero_and_inANDdecr_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(flat_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
flat_below_zero_and_inANDdecr_above_zero= list(set5)

flat_below_zero_and_mon_decr_above_zero= list(flat_below_zero_and_mon_decr_above_zero) + list(w3)
flat_below_zero_and_mon_incr_above_zero= list(flat_below_zero_and_mon_incr_above_zero) + list(w4)


In [101]:
i2= 0
flat_below_zero_and_mon_decr_and_stable_above_zero= list()
flat_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(flat_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in flat_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = flat_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                flat_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                flat_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(flat_below_zero_and_indecr_and_stable_above_zero)
set2= set(flat_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(flat_below_zero_and_mon_incr_and_stable_above_zero)
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
flat_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [102]:
w3= list()
w4= list()
i2=0

if len(flat_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(flat_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = flat_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [103]:
union = pd.Series(np.union1d(w3, w4))
flat_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(flat_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(flat_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

flat_below_zero_and_unclassified_above_zero= list(flat_below_zero_and_unclassified_above_zero)+list(flat_below_zero_and_unclassified_above_zero2)
flat_below_zero_and_unclassified_above_zero


[]

In [104]:
w4

set1= set(flat_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(flat_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
flat_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

flat_below_zero_and_mon_decr_and_stable_above_zero= list(flat_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
flat_below_zero_and_mon_incr_and_stable_above_zero= list(flat_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [105]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [106]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
        i2= i2+1

w3


[]

In [107]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)
unclassified_below_zero_and_flat_above_zero


[]

In [108]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_flat_above_zero= list(set5)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [109]:
i2= 0
mon_decr_and_stable_below_zero_and_flat_above_zero= list()
mon_incr_and_stable_below_zero_and_flat_above_zero= list()

if len(indecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_flat_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_flat_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_flat_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_flat_above_zero)
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_flat_above_zero


[]

In [110]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [111]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)+list(unclassified_below_zero_and_flat_above_zero2)
unclassified_below_zero_and_flat_above_zero


[]

In [112]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_flat_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_flat_above_zero= list(mon_decr_and_stable_below_zero_and_flat_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_flat_above_zero= list(mon_incr_and_stable_below_zero_and_flat_above_zero) + list(w4)


In [113]:
i2= 0
mon_decr_below_zero_and_flat_above_zero= list()
mon_incr_below_zero_and_flat_above_zero= list()

if len(diff_indecr_below_zero_and_flat_above_zero)>0:
    
    for r_id in diff_indecr_below_zero_and_flat_above_zero:
    
        r_id = diff_indecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_below_zero_and_flat_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_below_zero_and_flat_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(diff_indecr_below_zero_and_flat_above_zero)
set2= set(mon_decr_below_zero_and_flat_above_zero)
set3= set(mon_incr_below_zero_and_flat_above_zero)
inANDdecr_below_zero_and_flat_above_zero= list(set1-set2-set3)
inANDdecr_below_zero_and_flat_above_zero


[]

In [114]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_below_zero_and_flat_above_zero)>0:
    
    while i2 < len(inANDdecr_below_zero_and_flat_above_zero):
    
        i= 0
        r_id = inANDdecr_below_zero_and_flat_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [115]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_flat_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_flat_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_flat_above_zero= list(unclassified_below_zero_and_flat_above_zero)+list(unclassified_below_zero_and_flat_above_zero2)
unclassified_below_zero_and_flat_above_zero


[]

In [116]:
w4

set1= set(inANDdecr_below_zero_and_flat_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_flat_above_zero)
set5= set1-set2-set3-set4
inANDdecr_below_zero_and_flat_above_zero= list(set5)

mon_decr_below_zero_and_flat_above_zero= list(mon_decr_below_zero_and_flat_above_zero) + list(w3)
mon_incr_below_zero_and_flat_above_zero= list(mon_incr_below_zero_and_flat_above_zero) + list(w4)


In [117]:
i2= 0
mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list()

if len(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero


[]

In [118]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [119]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_indecr_and_stable_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_indecr_and_stable_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_indecr_and_stable_above_zero= list(unclassified_below_zero_and_indecr_and_stable_above_zero)+list(unclassified_below_zero_and_indecr_and_stable_above_zero2)
unclassified_below_zero_and_indecr_and_stable_above_zero


[]

In [120]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero) + list(w4)


In [121]:
i2= 0
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(inANDdecr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [122]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [123]:
union = pd.Series(np.union1d(w3, w4))
inANDdecr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [124]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [125]:
i2= 0
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_decr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [126]:
w3= list()
w4= list()
i2=0

if len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [127]:
union = pd.Series(np.union1d(w3, w4))
mon_decr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_decr_and_stable_below_zero_and_unclassified_above_zero= list(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
mon_decr_and_stable_below_zero_and_unclassified_above_zero


[]

In [128]:
w4

set1= set(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [129]:
i2= 0
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list()
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(mon_incr_and_stable_below_zero_and_indecr_and_stable_above_zero)
set2= set(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [130]:
w3= list()
w4= list()
i2=0

if len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [131]:
union = pd.Series(np.union1d(w3, w4))
mon_incr_and_stable_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

mon_incr_and_stable_below_zero_and_unclassified_above_zero= list(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
mon_incr_and_stable_below_zero_and_unclassified_above_zero


[]

In [132]:
w4

set1= set(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero= list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [133]:
i2= 0
mon_decr_and_stable_below_zero_and_unclassified_above_zero= list()
mon_incr_and_stable_below_zero_and_unclassified_above_zero= list()

if len(indecr_and_stable_below_zero_and_unclassified_above_zero)>0:
    
    for r_id in indecr_and_stable_below_zero_and_unclassified_above_zero:
    
        r_id = indecr_and_stable_below_zero_and_unclassified_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= 0
        l= list()
    
        while n2 < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                mon_decr_and_stable_below_zero_and_unclassified_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                mon_incr_and_stable_below_zero_and_unclassified_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(indecr_and_stable_below_zero_and_unclassified_above_zero)
set2= set(mon_decr_and_stable_below_zero_and_unclassified_above_zero)
set3= set(mon_incr_and_stable_below_zero_and_unclassified_above_zero)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(set1-set2-set3)
inANDdecr_and_stable_below_zero_and_unclassified_above_zero


[]

In [134]:
w3= list()
w4= list()
i2=0

if len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)>0:
    
    while i2 < len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero):
    
        i= 0
        r_id = inANDdecr_and_stable_below_zero_and_unclassified_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < int((len(csvFile)-1)/2+1):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [135]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_unclassified_above_zero = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_unclassified_above_zero)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_unclassified_above_zero)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_unclassified_above_zero= list(unclassified_below_zero_and_unclassified_above_zero)
unclassified_below_zero_and_unclassified_above_zero


[]

In [136]:
w4

set1= set(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
inANDdecr_and_stable_below_zero_and_unclassified_above_zero= list(set5)

mon_decr_and_stable_below_zero_and_unclassified_above_zero= list(mon_decr_and_stable_below_zero_and_unclassified_above_zero) + list(w3)
mon_incr_and_stable_below_zero_and_unclassified_above_zero= list(mon_incr_and_stable_below_zero_and_unclassified_above_zero) + list(w4)


In [137]:
i2= 0
unclassified_below_zero_and_mon_decr_and_stable_above_zero= list()
unclassified_below_zero_and_mon_incr_and_stable_above_zero= list()

if len(unclassified_below_zero_and_indecr_and_stable_above_zero)>0:
    
    for r_id in unclassified_below_zero_and_indecr_and_stable_above_zero:
    
        r_id = unclassified_below_zero_and_indecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        n2= int((len(csvFile)-1)/2)
        l= list()
    
        while n2 < len(csvFile):
            inf= csvFile.iloc[n2,  2]
            flt= float(csvFile.iloc[n2,  2])
    
            if csvFile.iloc[n2,  2] == flt:
                l.append(csvFile.iloc[n2,  2])
            n2= n2+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)

        my_series = csvFile2['Biomass'].squeeze()
    
        if len(csvFile2['Biomass'])>1:
        
            if my_series.is_monotonic_decreasing ==True:
                unclassified_below_zero_and_mon_decr_and_stable_above_zero.append(r_id)
            elif my_series.is_monotonic ==True:
                unclassified_below_zero_and_mon_incr_and_stable_above_zero.append(r_id)

    
        i2= i2 + 1

set1= set(unclassified_below_zero_and_indecr_and_stable_above_zero)
set2= set(unclassified_below_zero_and_mon_decr_and_stable_above_zero)
set3= set(unclassified_below_zero_and_mon_incr_and_stable_above_zero)
unclassified_below_zero_and_inANDdecr_and_stable_above_zero= list(set1-set2-set3)
unclassified_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [138]:
w3= list()
w4= list()
i2=0

if len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)>0:
    
    while i2 < len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero):
    
        i= int((len(csvFile)-1)/2)
        r_id = unclassified_below_zero_and_inANDdecr_and_stable_above_zero[i2]
        csvFile = pd.read_csv('biomass_per_flux_for_reaction_{}_lactose.csv'.format(r_id))
        l= list()

        while i < len(csvFile):
            inf= csvFile.iloc[i,  2]
            flt= float(csvFile.iloc[i,  2])
    
            if csvFile.iloc[i,  2] == flt:
                l.append(csvFile.iloc[i,  2])
            i= i+1

        csvFile2= pd.DataFrame (l, columns = ['Biomass'])
        csvFile2.Biomass = csvFile2.Biomass.round(2)
        l= list(csvFile2.Biomass)

        n2=0

        if l[n2] <max(l) and l[n2+1]<max(l) and l[n2+2]<max(l) and (l[n2+3] < max(l) or l[n2+3] == max(l)):
            n2=0
        elif l[n2] == max(l):
            n2=0
        else:
            w3.append(r_id)

    
        n2= len(l)-1

        if l[n2] <max(l) and l[n2-1]<max(l) and l[n2-2]<max(l) and (l[n2-3] < max(l) or l[n2-3] == max(l)):
            n2= len(l)-1
        elif l[n2] == max(l):
            n2= len(l)-1
        else:
            w4.append(r_id)
            
            
        i2= i2+1

w3


[]

In [139]:
union = pd.Series(np.union1d(w3, w4))
unclassified_below_zero_and_unclassified_above_zero2 = pd.Series(np.intersect1d(w3, w4))

set1= set(w3)
set2= set(unclassified_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w3= list(set3)

set1= set(w4)
set2= set(unclassified_below_zero_and_unclassified_above_zero2)
set3= set1-set2
w4= list(set3)

unclassified_below_zero_and_unclassified_above_zero= list(unclassified_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_unclassified_above_zero2)
unclassified_below_zero_and_unclassified_above_zero


[]

In [140]:
w4

set1= set(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)
set2= set(w3)
set3= set(w4)
set4= set(unclassified_below_zero_and_unclassified_above_zero)
set5= set1-set2-set3-set4
unclassified_below_zero_and_inANDdecr_and_stable_above_zero= list(set5)

unclassified_below_zero_and_mon_decr_and_stable_above_zero= list(unclassified_below_zero_and_mon_decr_and_stable_above_zero) + list(w3)
unclassified_below_zero_and_mon_incr_and_stable_above_zero= list(unclassified_below_zero_and_mon_incr_and_stable_above_zero) + list(w4)


In [141]:
d= pd.DataFrame (one, columns = ['Reaction names'])
d.to_csv("FINAL one feasible value NOT at zero reactions lactose.csv")
d1= pd.DataFrame (zero, columns = ['Reaction names'])
d1.to_csv("FINAL zero reactions lactose.csv")
D= pd.DataFrame (one_at_zero, columns = ['Reaction names'])
D.to_csv("FINAL one feasible value at zero reactions lactose.csv")
D1= pd.DataFrame (flat_above_zero, columns = ['Reaction names'])
D1.to_csv("FINAL flat above zero reactions lactose.csv")
D2= pd.DataFrame (flat_below_zero, columns = ['Reaction names'])
D2.to_csv("FINAL flat below zero reactions lactose.csv")
D3= pd.DataFrame (mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D3.to_csv("FINAL mon_decr_and_stable_above_zero reactions lactose.csv")
D4= pd.DataFrame (mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D4.to_csv("FINAL mon_incr_and_stable_above_zero reactions lactose.csv")
D5= pd.DataFrame (inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D5.to_csv("FINAL inANDdecr_and_stable_above_zero reactions lactose.csv")
D6= pd.DataFrame (mon_decr_and_stable_below_zero, columns = ['Reaction names'])
D6.to_csv("FINAL increasing and stable below zero reactions lactose.csv")
D7= pd.DataFrame (mon_incr_and_stable_below_zero, columns = ['Reaction names'])
D7.to_csv("FINAL decreasing and stable below reactions lactose.csv")
D8= pd.DataFrame (inANDdecr_and_stable_below_zero, columns = ['Reaction names'])
D8.to_csv("FINAL inANDdecr_and_stable_below_zero reactions lactose.csv")
D9= pd.DataFrame (decr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D9.to_csv("FINAL increasing and stable below zero and one to three data point above zero reactions lactose.csv")
D10= pd.DataFrame (incr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D10.to_csv("FINAL decreasing and stable below zero and one to three data point above zero reactions lactose.csv")
D11= pd.DataFrame (inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D11.to_csv("FINAL inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero reactions lactose.csv")
D12= pd.DataFrame (decr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D12.to_csv("FINAL increasing and stable below zero and decreasing above zero reactions lactose.csv")
D13= pd.DataFrame (incr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D13.to_csv("FINAL decreasing and stable below zero and decreasing above zero reactions lactose.csv")
D14= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D14.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_decr_above_zero reactions lactose.csv")
D15= pd.DataFrame (decr_and_stable_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D15.to_csv("FINAL increasing and stable below zero and increasing above zero reactions lactose.csv")
D16= pd.DataFrame (incr_and_stable_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D16.to_csv("FINAL decreasing and stable below zero and increasing above zero reactions lactose.csv")
D17= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D17.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_incr_above_zero reactions lactose.csv")
D18= pd.DataFrame (decr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D18.to_csv("FINAL increasing and stable below zero and increasing and decreasing above zero reactions lactose.csv")
D19= pd.DataFrame (incr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D19.to_csv("FINAL decreasing and stable below zero and increasing and decreasing above zero reactions lactose.csv")
D20= pd.DataFrame (inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D20.to_csv("FINAL inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero reactions lactose.csv")
D21= pd.DataFrame (one_to_three_data_point_below_zero_and_decr_and_stable_above_zero, columns = ['Reaction names'])
D21.to_csv("FINAL one_to_three_data_point_below_zero_and_decr_and_stable_above_zero reactions lactose.csv")
D22= pd.DataFrame (one_to_three_data_point_below_zero_and_incr_and_stable_above_zero, columns = ['Reaction names'])
D22.to_csv("FINAL one_to_three_data_point_below_zero_and_incr_and_stable_above_zero reactions lactose.csv")
D23= pd.DataFrame (one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D23.to_csv("FINAL one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero reactions lactose.csv")
D24= pd.DataFrame (one_to_three_data_point_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D24.to_csv("FINAL one_to_three_data_point_below_zero_and_mon_decr_above_zero reactions lactose.csv")
D25= pd.DataFrame (one_to_three_data_point_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D25.to_csv("FINAL one_to_three_data_point_below_zero_and_mon_incr_above_zero reactions lactose.csv")
D26= pd.DataFrame (one_to_three_data_point_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D26.to_csv("FINAL one_to_three_data_point_below_zero_and_inANDdecr_above_zero reactions lactose.csv")
D27= pd.DataFrame (inANDdecr_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D27.to_csv("FINAL inANDdecr_below_zero_and_mon_decr_above_zero reactions lactose.csv")
D28= pd.DataFrame (mon_decr_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D28.to_csv("FINAL increasing below zero decreasing above zero reactions lactose.csv")
D29= pd.DataFrame (mon_incr_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D29.to_csv("FINAL decreasing below zero decreasing above zero reactions lactose.csv")
D30= pd.DataFrame (inANDdecr_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D30.to_csv("FINAL inANDdecr_below_zero_and_mon_incr_above_zero reactions lactose.csv")
D31= pd.DataFrame (mon_decr_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D31.to_csv("FINAL increasing below zero and increasing above zero reactions lactose.csv")
D32= pd.DataFrame (mon_incr_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D32.to_csv("FINAL decreasing below zero and increasing above zero reactions lactose.csv")
D33= pd.DataFrame (inANDdecr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D33.to_csv("FINAL inANDdecr_below_zero_and_inANDdecr_above_zero reactions lactose.csv")
D34= pd.DataFrame (mon_decr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D34.to_csv("FINAL increasing below zero and increasing and decreasing above zero reactions lactose.csv")
D35= pd.DataFrame (mon_incr_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D35.to_csv("FINAL decreasing below zero and increasing and decreasing above zero reactions lactose.csv")
D36= pd.DataFrame (inANDdecr_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D36.to_csv("FINAL inANDdecr_below_zero_and_mon_decr_and_stable_above_zero reactions lactose.csv")
D37= pd.DataFrame (inANDdecr_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D37.to_csv("FINAL inANDdecr_below_zero_and_mon_incr_and_stable_above_zero reactions lactose.csv")
D38= pd.DataFrame (inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D38.to_csv("FINAL inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero reactions lactose.csv")
D39= pd.DataFrame (mon_decr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D39.to_csv("FINAL increasing below zero and increasing and decreasing and stable above zero reactions lactose.csv")
D40= pd.DataFrame (mon_decr_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D40.to_csv("FINAL increasing below zero and decreasing and stable above zero reactions lactose.csv")
D41= pd.DataFrame (mon_decr_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D41.to_csv("FINAL increasing below zero and increasing and stable above zero reactions lactose.csv")
D42= pd.DataFrame (mon_incr_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D42.to_csv("FINAL decreasing below zero and increasing and decreasing and stable above zero reactions lactose.csv")
D43= pd.DataFrame (mon_incr_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D43.to_csv("FINAL decreasing below zero and decreasing and stable above zero reactions lactose.csv")
D44= pd.DataFrame (mon_incr_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D44.to_csv("FINAL decreasing below zero and increasing and stable above zero reactions lactose.csv")
D45= pd.DataFrame (inANDdecr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D45.to_csv("FINAL inANDdecr_below_zero_and_one_to_three_data_point_above_zero reactions lactose.csv")
D46= pd.DataFrame (mon_decr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D46.to_csv("FINAL increasing below zero and one to three data point above zero reactions lactose.csv")
D47= pd.DataFrame (mon_incr_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D47.to_csv("FINAL decreasing below zero and one to three data point above zero reactions lactose.csv")
D48= pd.DataFrame (flat_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D48.to_csv("FINAL flat_below_zero_and_inANDdecr_above_zero reactions lactose.csv")
D49= pd.DataFrame (flat_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D49.to_csv("FINAL flat_below_zero_and_mon_decr_above_zero reactions lactose.csv")
D50= pd.DataFrame (flat_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D50.to_csv("FINAL flat_below_zero_and_mon_incr_above_zero reactions lactose.csv")
D51= pd.DataFrame (flat_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D51.to_csv("FINAL flat_below_zero_and_inANDdecr_and_stable_above_zero reactions lactose.csv")
D52= pd.DataFrame (flat_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D52.to_csv("FINAL flat_below_zero_and_mon_decr_and_stable_above_zero reactions lactose.csv")
D53= pd.DataFrame (flat_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D53.to_csv("FINAL flat_below_zero_and_mon_incr_and_stable_above_zero reactions lactose.csv")
D54= pd.DataFrame (inANDdecr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D54.to_csv("FINAL inANDdecr_and_stable_below_zero_and_flat_above_zero reactions lactose.csv")
D55= pd.DataFrame (mon_decr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D55.to_csv("FINAL increasing and stable below zero and flat above zero reactions lactose.csv")
D56= pd.DataFrame (mon_incr_and_stable_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D56.to_csv("FINAL decreasing and stable below zero and flat above zero reactions lactose.csv")
D57= pd.DataFrame (inANDdecr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D57.to_csv("FINAL inANDdecr_below_zero_and_flat_above_zero reactions lactose.csv")
D58= pd.DataFrame (mon_decr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D58.to_csv("FINAL increasing below zero and flat above zero reactions lactose.csv")
D59= pd.DataFrame (mon_incr_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D59.to_csv("FINAL decreasing below zero and flat above zero reactions lactose.csv")
D60= pd.DataFrame (inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D60.to_csv("FINAL inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero reactions lactose.csv")
D61= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D61.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero reactions lactose.csv")
D62= pd.DataFrame (inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D62.to_csv("FINAL inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero reactions lactose.csv")
D63= pd.DataFrame (mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D63.to_csv("FINAL increasing and stable below zero and increasing and decreasing and stable above zero reactions lactose.csv")
D64= pd.DataFrame (mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D64.to_csv("FINAL increasing and stable below zero and decreasing and stable above zero reactions lactose.csv")
D65= pd.DataFrame (mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D65.to_csv("FINAL increasing and stable below zero and increasing and stable above zero reactions lactose.csv")
D66= pd.DataFrame (mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D66.to_csv("FINAL decreasing and stable below zero and increasing and decreasing and stable above zero reactions lactose.csv")
D67= pd.DataFrame (mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D67.to_csv("FINAL decreasing and stable below zero and decreasing and stable above zero reactions lactose.csv")
D68= pd.DataFrame (mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D68.to_csv("FINAL decreasing and stable below zero and increasing and stable above zero reactions lactose.csv")
D69= pd.DataFrame (flat_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D69.to_csv("FINAL flat_below_zero_and_flat_above_zero reactions lactose.csv")
D70= pd.DataFrame (one_to_three_data_point_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D70.to_csv("FINAL one_to_three_data_point_below_zero_and_flat_above_zero reactions lactose.csv")
D71= pd.DataFrame (one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D71.to_csv("FINAL one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero reactions lactose.csv")
D72= pd.DataFrame (flat_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D72.to_csv("FINAL flat_below_zero_and_one_to_three_data_point_above_zero reactions lactose.csv")
D73= pd.DataFrame (unclassified_above_zero, columns = ['Reaction names'])
D73.to_csv("FINAL unclassified_above_zero reactions lactose.csv")
D74= pd.DataFrame (unclassified_below_zero, columns = ['Reaction names'])
D74.to_csv("FINAL unclassified_below_zero reactions lactose.csv")
D75= pd.DataFrame (one_to_three_data_point_above_zero, columns = ['Reaction names'])
D75.to_csv("FINAL one_to_three_data_point_above_zero reactions lactose.csv")
D76= pd.DataFrame (one_to_three_data_point_below_zero, columns = ['Reaction names'])
D76.to_csv("FINAL one_to_three_data_point_below_zero reactions lactose.csv")
D77= pd.DataFrame (mon_decr_above_zero, columns = ['Reaction names'])
D77.to_csv("FINAL mon_decr_above_zero reactions lactose.csv")
D78= pd.DataFrame (mon_incr_above_zero, columns = ['Reaction names'])
D78.to_csv("FINAL mon_incr_above_zero reactions lactose.csv")
D79= pd.DataFrame (inANDdecr_above_zero, columns = ['Reaction names'])
D79.to_csv("FINAL inANDdecr_above_zero reactions lactose.csv")
D80= pd.DataFrame (mon_decr_below_zero, columns = ['Reaction names'])
D80.to_csv("FINAL increasing below zero reactions lactose.csv")
D81= pd.DataFrame (mon_incr_below_zero, columns = ['Reaction names'])
D81.to_csv("FINAL decreasing below zero reactions lactose.csv")
D82= pd.DataFrame (inANDdecr_below_zero, columns = ['Reaction names'])
D82.to_csv("FINAL inANDdecr_below_zero reactions lactose.csv")
D83= pd.DataFrame (unclassified_below_zero_and_one_to_three_data_point_above_zero, columns = ['Reaction names'])
D83.to_csv("FINAL unclassified_below_zero_and_one_to_three_data_point_above_zero reactions lactose.csv")
D84= pd.DataFrame (unclassified_below_zero_and_mon_decr_above_zero, columns = ['Reaction names'])
D84.to_csv("FINAL unclassified_below_zero_and_mon_decr_above_zero reactions lactose.csv")
D85= pd.DataFrame (unclassified_below_zero_and_mon_incr_above_zero, columns = ['Reaction names'])
D85.to_csv("FINAL unclassified_below_zero_and_mon_incr_above_zero reactions lactose.csv")
D86= pd.DataFrame (unclassified_below_zero_and_inANDdecr_above_zero, columns = ['Reaction names'])
D86.to_csv("FINAL unclassified_below_zero_and_inANDdecr_above_zero reactions lactose.csv")
D87= pd.DataFrame (one_to_three_data_point_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D87.to_csv("FINAL one_to_three_data_point_below_zero_and_unclassified_above_zero reactions lactose.csv")
D88= pd.DataFrame (inANDdecr_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D88.to_csv("FINAL inANDdecr_below_zero_and_unclassified_above_zero reactions lactose.csv")
D89= pd.DataFrame (mon_decr_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D89.to_csv("FINAL increasing below zero and unclassified above zero reactions lactose.csv")
D90= pd.DataFrame (mon_incr_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D90.to_csv("FINAL decreasing below zero and unclassified above zero reactions lactose.csv")
D91= pd.DataFrame (flat_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D91.to_csv("FINAL flat_below_zero_and_unclassified_above_zero reactions lactose.csv")
D92= pd.DataFrame (unclassified_below_zero_and_flat_above_zero, columns = ['Reaction names'])
D92.to_csv("FINAL unclassified_below_zero_and_flat_above_zero reactions lactose.csv")
D93= pd.DataFrame (inANDdecr_and_stable_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D93.to_csv("FINAL inANDdecr_and_stable_below_zero_and_unclassified_above_zero reactions lactose.csv")
D94= pd.DataFrame (mon_decr_and_stable_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D94.to_csv("FINAL increasing and stable below zero and unclassified above zero reactions lactose.csv")
D95= pd.DataFrame (mon_incr_and_stable_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D95.to_csv("FINAL decreasing and stable below zero and unclassified above zero reactions lactose.csv")
D96= pd.DataFrame (unclassified_below_zero_and_unclassified_above_zero, columns = ['Reaction names'])
D96.to_csv("FINAL unclassified_below_zero_and_unclassified_above_zero reactions lactose.csv")
D97= pd.DataFrame (unclassified_below_zero_and_mon_decr_and_stable_above_zero, columns = ['Reaction names'])
D97.to_csv("FINAL unclassified_below_zero_and_mon_decr_and_stable_above_zero reactions lactose.csv")
D98= pd.DataFrame (unclassified_below_zero_and_mon_incr_and_stable_above_zero, columns = ['Reaction names'])
D98.to_csv("FINAL unclassified_below_zero_and_mon_incr_and_stable_above_zero reactions lactose.csv")
D99= pd.DataFrame (unclassified_below_zero_and_inANDdecr_and_stable_above_zero, columns = ['Reaction names'])
D99.to_csv("FINAL unclassified_below_zero_and_inANDdecr_and_stable_above_zero reactions lactose.csv")


In [142]:
len(inANDdecr_and_stable_below_zero)

19

In [143]:
one
zero
one_at_zero
flat_above_zero
flat_below_zero

mon_decr_and_stable_above_zero
mon_incr_and_stable_above_zero
inANDdecr_and_stable_above_zero

mon_decr_and_stable_below_zero
mon_incr_and_stable_below_zero
inANDdecr_and_stable_below_zero

decr_and_stable_below_zero_and_one_to_three_data_point_above_zero
incr_and_stable_below_zero_and_one_to_three_data_point_above_zero
inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero

decr_and_stable_below_zero_and_mon_decr_above_zero
incr_and_stable_below_zero_and_mon_decr_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_above_zero

decr_and_stable_below_zero_and_mon_incr_above_zero
incr_and_stable_below_zero_and_mon_incr_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_above_zero

decr_and_stable_below_zero_and_inANDdecr_above_zero
incr_and_stable_below_zero_and_inANDdecr_above_zero
inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_decr_and_stable_above_zero
one_to_three_data_point_below_zero_and_incr_and_stable_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero

one_to_three_data_point_below_zero_and_mon_decr_above_zero
one_to_three_data_point_below_zero_and_mon_incr_above_zero
one_to_three_data_point_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_above_zero
mon_decr_below_zero_and_mon_decr_above_zero
mon_incr_below_zero_and_mon_decr_above_zero

inANDdecr_below_zero_and_mon_incr_above_zero
mon_decr_below_zero_and_mon_incr_above_zero
mon_incr_below_zero_and_mon_incr_above_zero

inANDdecr_below_zero_and_inANDdecr_above_zero
mon_decr_below_zero_and_inANDdecr_above_zero
mon_incr_below_zero_and_inANDdecr_above_zero

inANDdecr_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_below_zero_and_mon_incr_and_stable_above_zero
inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero

mon_decr_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_below_zero_and_one_to_three_data_point_above_zero
mon_decr_below_zero_and_one_to_three_data_point_above_zero
mon_incr_below_zero_and_one_to_three_data_point_above_zero

flat_below_zero_and_inANDdecr_above_zero
flat_below_zero_and_mon_decr_above_zero
flat_below_zero_and_mon_incr_above_zero

flat_below_zero_and_inANDdecr_and_stable_above_zero
flat_below_zero_and_mon_decr_and_stable_above_zero
flat_below_zero_and_mon_incr_and_stable_above_zero

inANDdecr_and_stable_below_zero_and_flat_above_zero
mon_decr_and_stable_below_zero_and_flat_above_zero
mon_incr_and_stable_below_zero_and_flat_above_zero

inANDdecr_below_zero_and_flat_above_zero
mon_decr_below_zero_and_flat_above_zero
mon_incr_below_zero_and_flat_above_zero

inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero
mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero

flat_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_flat_above_zero
one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero
flat_below_zero_and_one_to_three_data_point_above_zero

unclassified_above_zero
unclassified_below_zero
one_to_three_data_point_above_zero
one_to_three_data_point_below_zero

mon_decr_above_zero
mon_incr_above_zero
inANDdecr_above_zero

mon_decr_below_zero
mon_incr_below_zero
inANDdecr_below_zero

unclassified_below_zero_and_one_to_three_data_point_above_zero
unclassified_below_zero_and_mon_decr_above_zero
unclassified_below_zero_and_mon_incr_above_zero
unclassified_below_zero_and_inANDdecr_above_zero

one_to_three_data_point_below_zero_and_unclassified_above_zero
inANDdecr_below_zero_and_unclassified_above_zero
mon_decr_below_zero_and_unclassified_above_zero
mon_incr_below_zero_and_unclassified_above_zero

flat_below_zero_and_unclassified_above_zero
unclassified_below_zero_and_flat_above_zero
inANDdecr_and_stable_below_zero_and_unclassified_above_zero
mon_decr_and_stable_below_zero_and_unclassified_above_zero
mon_incr_and_stable_below_zero_and_unclassified_above_zero

mon_decr_and_stable_below_zero_and_unclassified_above_zero
mon_incr_and_stable_below_zero_and_unclassified_above_zero
inANDdecr_and_stable_below_zero_and_unclassified_above_zero
unclassified_below_zero_and_unclassified_above_zero

unclassified_below_zero_and_mon_decr_and_stable_above_zero
unclassified_below_zero_and_mon_incr_and_stable_above_zero
unclassified_below_zero_and_inANDdecr_and_stable_above_zero


[]

In [144]:
len(flat_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_flat_above_zero)+ len(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ len(flat_below_zero_and_one_to_three_data_point_above_zero)+ len(one_at_zero)+len(one)+len(zero)+len(mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero)+len(mon_incr_and_stable_below_zero)+len(inANDdecr_and_stable_below_zero)+len(flat_above_zero)+len(flat_below_zero)+len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)+len(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+len(decr_and_stable_below_zero_and_mon_decr_above_zero)+len(incr_and_stable_below_zero_and_mon_decr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+len(decr_and_stable_below_zero_and_mon_incr_above_zero)+len(incr_and_stable_below_zero_and_mon_incr_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+len(decr_and_stable_below_zero_and_inANDdecr_above_zero)+len(incr_and_stable_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+len(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+len(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+len(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_above_zero)+len(mon_decr_below_zero_and_mon_decr_above_zero)+len(mon_incr_below_zero_and_mon_decr_above_zero)+len(inANDdecr_below_zero_and_mon_incr_above_zero)+len(mon_decr_below_zero_and_mon_incr_above_zero)+len(mon_incr_below_zero_and_mon_incr_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_above_zero)+len(mon_decr_below_zero_and_inANDdecr_above_zero)+len(mon_incr_below_zero_and_inANDdecr_above_zero)+len(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+len(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+len(flat_below_zero_and_inANDdecr_above_zero)+len(flat_below_zero_and_mon_decr_above_zero)+len(flat_below_zero_and_mon_incr_above_zero)+len(flat_below_zero_and_inANDdecr_and_stable_above_zero)+len(flat_below_zero_and_mon_decr_and_stable_above_zero)+len(flat_below_zero_and_mon_incr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_flat_above_zero)+len(mon_decr_and_stable_below_zero_and_flat_above_zero)+len(mon_incr_and_stable_below_zero_and_flat_above_zero)+len(inANDdecr_below_zero_and_flat_above_zero)+len(mon_decr_below_zero_and_flat_above_zero)+len(mon_incr_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+len(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+len(unclassified_above_zero)+len(unclassified_below_zero)+len(one_to_three_data_point_below_zero)+len(one_to_three_data_point_above_zero)+len(unclassified_below_zero_and_one_to_three_data_point_above_zero)+len(unclassified_below_zero_and_mon_decr_above_zero)+len(unclassified_below_zero_and_mon_incr_above_zero)+len(unclassified_below_zero_and_inANDdecr_above_zero)+len(one_to_three_data_point_below_zero_and_unclassified_above_zero)+len(inANDdecr_below_zero_and_unclassified_above_zero)+len(mon_decr_below_zero_and_unclassified_above_zero)+len(mon_incr_below_zero_and_unclassified_above_zero)+len(flat_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_flat_above_zero)+len(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)+len(mon_decr_and_stable_below_zero_and_unclassified_above_zero)+len(mon_incr_and_stable_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_unclassified_above_zero)+len(unclassified_below_zero_and_mon_decr_and_stable_above_zero)+len(unclassified_below_zero_and_mon_incr_and_stable_above_zero)+len(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)


2726

In [145]:
lt= list(flat_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_flat_above_zero)+ list(one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero)+ list(flat_below_zero_and_one_to_three_data_point_above_zero)+ list(one_at_zero)+list(one)+list(zero)+list(mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero)+list(mon_incr_and_stable_below_zero)+list(inANDdecr_and_stable_below_zero)+list(flat_above_zero)+list(flat_below_zero)+list(mon_decr_above_zero)+list(mon_incr_above_zero)+list(inANDdecr_above_zero)+list(mon_decr_below_zero)+list(mon_incr_below_zero)+list(inANDdecr_below_zero)+list(decr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(incr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero)+list(decr_and_stable_below_zero_and_mon_decr_above_zero)+list(incr_and_stable_below_zero_and_mon_decr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_above_zero)+list(decr_and_stable_below_zero_and_mon_incr_above_zero)+list(incr_and_stable_below_zero_and_mon_incr_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_above_zero)+list(decr_and_stable_below_zero_and_inANDdecr_above_zero)+list(incr_and_stable_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_decr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_incr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero)+list(one_to_three_data_point_below_zero_and_mon_decr_above_zero)+list(one_to_three_data_point_below_zero_and_mon_incr_above_zero)+list(one_to_three_data_point_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_above_zero)+list(mon_decr_below_zero_and_mon_decr_above_zero)+list(mon_incr_below_zero_and_mon_decr_above_zero)+list(inANDdecr_below_zero_and_mon_incr_above_zero)+list(mon_decr_below_zero_and_mon_incr_above_zero)+list(mon_incr_below_zero_and_mon_incr_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_above_zero)+list(mon_decr_below_zero_and_inANDdecr_above_zero)+list(mon_incr_below_zero_and_inANDdecr_above_zero)+list(inANDdecr_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_decr_below_zero_and_one_to_three_data_point_above_zero)+list(mon_incr_below_zero_and_one_to_three_data_point_above_zero)+list(flat_below_zero_and_inANDdecr_above_zero)+list(flat_below_zero_and_mon_decr_above_zero)+list(flat_below_zero_and_mon_incr_above_zero)+list(flat_below_zero_and_inANDdecr_and_stable_above_zero)+list(flat_below_zero_and_mon_decr_and_stable_above_zero)+list(flat_below_zero_and_mon_incr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_flat_above_zero)+list(mon_decr_and_stable_below_zero_and_flat_above_zero)+list(mon_incr_and_stable_below_zero_and_flat_above_zero)+list(inANDdecr_below_zero_and_flat_above_zero)+list(mon_decr_below_zero_and_flat_above_zero)+list(mon_incr_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero)+list(mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero)+list(unclassified_above_zero)+list(unclassified_below_zero)+list(one_to_three_data_point_below_zero)+list(one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_one_to_three_data_point_above_zero)+list(unclassified_below_zero_and_mon_decr_above_zero)+list(unclassified_below_zero_and_mon_incr_above_zero)+list(unclassified_below_zero_and_inANDdecr_above_zero)+list(one_to_three_data_point_below_zero_and_unclassified_above_zero)+list(inANDdecr_below_zero_and_unclassified_above_zero)+list(mon_decr_below_zero_and_unclassified_above_zero)+list(mon_incr_below_zero_and_unclassified_above_zero)+list(flat_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_flat_above_zero)+list(inANDdecr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_decr_and_stable_below_zero_and_unclassified_above_zero)+list(mon_incr_and_stable_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_unclassified_above_zero)+list(unclassified_below_zero_and_mon_decr_and_stable_above_zero)+list(unclassified_below_zero_and_mon_incr_and_stable_above_zero)+list(unclassified_below_zero_and_inANDdecr_and_stable_above_zero)


In [146]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [147]:
csvFile = pd.read_csv('all reactions lactose.csv')
csvFile["Reaction names"]

rl= list(csvFile["Reaction names"])

set1= set(rl)
set2= set(lt)
set3= set1-set2
set3


set()

In [148]:
len(diff_indecr_above_zero)+len(diff_indecr_below_zero)


736

In [149]:
len(mon_decr_above_zero)+len(mon_incr_above_zero)+len(inANDdecr_above_zero)+len(mon_decr_below_zero)+len(mon_incr_below_zero)+len(inANDdecr_below_zero)+len(unclassified_above_zero)+len(unclassified_below_zero)


736

In [150]:
lt= list(mon_decr_above_zero)+list(mon_incr_above_zero)+list(inANDdecr_above_zero)+list(mon_decr_below_zero)+list(mon_incr_below_zero)+list(inANDdecr_below_zero)+list(unclassified_above_zero)+list(unclassified_below_zero)


In [151]:
l1=[]
for i in lt:
    if i not in l1:
        l1.append(i)
    else:
        print(i,end=' ')
        

In [317]:
i= 0
csvFile = pd.read_csv('FINAL different reactions glu vs lac.csv')
csvFile["Category in lactose"] = ""
csvFile.iloc[0,3]

''

In [318]:
i= 0
csvFile = pd.read_csv('FINAL different reactions glu vs lac.csv')
csvFile["Category in lactose"] = ""
csvFile

while i < len(csvFile):
    if csvFile.iloc[i, 1] in one:
        csvFile.iloc[i, 3] = "one"
    if csvFile.iloc[i, 1] in zero:
        csvFile.iloc[i, 3] = "zero"
    if csvFile.iloc[i, 1] in one_at_zero:
        csvFile.iloc[i, 3] = "one_at_zero"
    if csvFile.iloc[i, 1] in flat_above_zero:
        csvFile.iloc[i, 3] = "flat_above_zero"
    if csvFile.iloc[i, 1] in flat_below_zero:
        csvFile.iloc[i, 3] = "flat_below_zero"
    if csvFile.iloc[i, 1] in mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "mon_incr_and_stable_above_zero"   
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_and_stable_below_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero"
    if csvFile.iloc[i, 1] in mon_incr_and_stable_below_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero"
    if csvFile.iloc[i, 1] in decr_and_stable_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and one to three data point above zero"
    if csvFile.iloc[i, 1] in incr_and_stable_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and one to three data point above zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 1] in decr_and_stable_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and decreasing above zero"
    if csvFile.iloc[i, 1] in incr_and_stable_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and decreasing above zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_mon_decr_above_zero"   
    if csvFile.iloc[i, 1] in decr_and_stable_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and increasing above zero"
    if csvFile.iloc[i, 1] in incr_and_stable_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and increasing above zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 1] in decr_and_stable_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and increasing and decreasing above zero"
    if csvFile.iloc[i, 1] in incr_and_stable_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and increasing and decreasing above zero"   
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_decr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_incr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and decreasing above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and decreasing above zero"   
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and increasing above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and increasing above zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and increasing and decreasing above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and increasing and decreasing above zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_mon_incr_and_stable_above_zero"   
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and increasing and decreasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and increasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and increasing and decreasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and increasing and stable above zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and one to three data point above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and one to three data point above zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_mon_incr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_and_stable_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and flat above zero"
    if csvFile.iloc[i, 1] in mon_incr_and_stable_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and flat above zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and flat above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and flat above zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_mon_incr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and inANDdecr_and_stable above zero"
    if csvFile.iloc[i, 1] in mon_decr_and_stable_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_decr_and_stable_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and increasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_incr_and_stable_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and inANDdecr_and_stable above zero"
    if csvFile.iloc[i, 1] in mon_incr_and_stable_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and decreasing and stable above zero"
    if csvFile.iloc[i, 1] in mon_incr_and_stable_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and increasing and stable above zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 1] in unclassified_above_zero:
        csvFile.iloc[i, 3] = "unclassified_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero"
    if csvFile.iloc[i, 1] in mon_decr_above_zero:
        csvFile.iloc[i, 3] = "mon_decr_above_zero"
    if csvFile.iloc[i, 1] in mon_incr_above_zero:
        csvFile.iloc[i, 3] = "mon_incr_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero:
        csvFile.iloc[i, 3] = "increasing below zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero:
        csvFile.iloc[i, 3] = "decreasing below zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_one_to_three_data_point_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_one_to_three_data_point_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_mon_decr_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_mon_decr_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_mon_incr_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_mon_incr_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_inANDdecr_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_inANDdecr_above_zero"
    if csvFile.iloc[i, 1] in one_to_three_data_point_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "one_to_three_data_point_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "increasing below zero and unclassified above zero"
    if csvFile.iloc[i, 1] in mon_incr_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "decreasing below zero and unclassified above zero"
    if csvFile.iloc[i, 1] in flat_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "flat_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_flat_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_flat_above_zero"
    if csvFile.iloc[i, 1] in inANDdecr_and_stable_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "inANDdecr_and_stable_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 1] in mon_decr_and_stable_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "increasing and stable below zero and unclassified above zero"
    if csvFile.iloc[i, 1] in mon_incr_and_stable_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "decreasing and stable below zero and unclassified above zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_unclassified_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_unclassified_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_mon_decr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_mon_decr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_mon_incr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_mon_incr_and_stable_above_zero"
    if csvFile.iloc[i, 1] in unclassified_below_zero_and_inANDdecr_and_stable_above_zero:
        csvFile.iloc[i, 3] = "unclassified_below_zero_and_inANDdecr_and_stable_above_zero"

    i= i+1

csvFile
csvFile.to_csv("FINAL different reactions glu vs lac.csv")


In [320]:
csvFile.iloc[0, 3]

'mon_decr_above_zero'

In [321]:
csvFile

,Unnamed: 0,Reaction names,Category in glucose,Category in lactose
0,0,ACOAD4f,one_to_three_data_point_above_zero,mon_decr_above_zero
1,1,H2Stex,one_to_three_data_point_below_zero,decreasing below zero
2,2,PAPSR,decreasing below zero and one to three data po...,decreasing below zero and decreasing above zero
3,3,ARGAGMt7pp,decreasing below zero,decreasing below zero and one to three data po...
4,4,METAT,one_at_zero,one_to_three_data_point_above_zero
...,...,...,...,...
417,417,DHORTS,one_to_three_data_point_below_zero,unclassified_below_zero
418,418,FACOAE141,one_to_three_data_point_below_zero_and_mon_dec...,decreasing below zero and decreasing above zero
419,419,ACACT3r,flat_above_zero,one_to_three_data_point_below_zero_and_flat_ab...
420,420,EX_glc__D_e,inANDdecr_and_stable_below_zero,inANDdecr_and_stable_below_zero_and_one_to_thr...
